### CNN
* http://andrew.gibiansky.com/blog/machine-learning/convolutional-neural-networks/
* http://www.wildml.com/2015/11/understanding-convolutional-neural-networks-for-nlp/
* https://medium.com/@ageitgey/machine-learning-is-fun-part-3-deep-learning-and-convolutional-neural-networks-f40359318721
* https://algotravelling.com/ru/%D0%BC%D0%B0%D1%88%D0%B8%D0%BD%D0%BD%D0%BE%D0%B5-%D0%BE%D0%B1%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D0%B5-%D1%8D%D1%82%D0%BE-%D0%B2%D0%B5%D1%81%D0%B5%D0%BB%D0%BE-3/
* https://adeshpande3.github.io/adeshpande3.github.io/A-Beginner's-Guide-To-Understanding-Convolutional-Neural-Networks-Part-2/
* https://medium.com/@karpathy/yes-you-should-understand-backprop-e2f06eab496b
* https://www.asozykin.ru/courses/nnpython
* https://www.tensorflow.org/tutorials/deep_cnn#convolutional-neural-networks

## Papers
* https://arxiv.org/pdf/1512.03385.pdf - resnet
* https://arxiv.org/pdf/1409.1556.pdf - vgg16
* https://arxiv.org/pdf/1512.00567.pdf - inception v3

In [11]:
from scipy import ndimage
import numpy as np
import pandas as pd
import os
from IPython.display import display, Image
from scipy import misc
import matplotlib.pyplot as plt
import tensorflow
import seaborn

from sklearn.model_selection import train_test_split

from keras.models import Sequential, load_model, Model
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense, BatchNormalization, GlobalAveragePooling2D, Input
from keras.optimizers import SGD, Adam
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping, ReduceLROnPlateau
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import InceptionV3

%matplotlib inline

### Keras

Виктор
* арима для каждого игрока
* последние 2 месяца
* фб профит либа

* уменьшить число фич - можно побить бейслайн

vgg для 224\224
keras vgg
уменьшить количество каналов в 4 раза
resnet: первый слов свертка 7х7 пулинг 3х3 убрать
конва + идентити блок




In [29]:
img_width = img_height = 150
img_channels = 3

In [3]:
def add_conv2d(model, kernel_size, filter_size = (3, 3), padding='same', activation='relu'):
    model.add(Conv2D(kernel_size, filter_size, padding=padding))
    model.add(BatchNormalization())
    model.add(Activation(activation))
    
def add_dens(model, kernel_size, activation='relu'):
    model.add(Dense(kernel_size))
    model.add(BatchNormalization())
    model.add(Activation(activation))

# VGG16
https://arxiv.org/pdf/1409.1556.pdf - vgg16

In [ ]:
first = 32
second = 64
third = 128
fourth = fifth = 256
filter_size = (3,3)

model = Sequential()
# first
model.add(Conv2D(first, filter_size, padding='same', input_shape=(img_width, img_height, img_channels)))
model.add(BatchNormalization())
model.add(Activation('relu'))
add_conv2d(model, first)
model.add(MaxPooling2D(pool_size = (2, 2), strides=(2, 2)))

# second
add_conv2d(model, second)
add_conv2d(model, second)
model.add(MaxPooling2D(pool_size = (2, 2), strides=(2, 2)))

# third
add_conv2d(model, third)
add_conv2d(model, third)
add_conv2d(model, third)
model.add(MaxPooling2D(pool_size = (2, 2), strides=(2, 2)))

# fourth
add_conv2d(model, fourth)
add_conv2d(model, fourth)
add_conv2d(model, fourth)
model.add(MaxPooling2D(pool_size = (2, 2), strides=(2, 2)))

# fifth
add_conv2d(model, fifth)
add_conv2d(model, fifth)
add_conv2d(model, fifth)
model.add(MaxPooling2D(pool_size = (2, 2), strides=(2, 2)))

model.add(GlobalAveragePooling2D())
add_dens(model, 2048)
add_dens(model, 2048)
add_dens(model, 256, activation='softmax')

# VGG19

In [ ]:
first = 32
second = 64
third = 128
fourth = fifth = 256
filter_size = (3,3)

model = Sequential()
# first
model.add(Conv2D(first, filter_size, padding='same', input_shape=(img_width, img_height, img_channels)))
model.add(BatchNormalization())
model.add(Activation('relu'))
add_conv2d(model, first)
model.add(MaxPooling2D(pool_size = (2, 2), strides=(2, 2)))

# second
add_conv2d(model, second)
add_conv2d(model, second)
model.add(MaxPooling2D(pool_size = (2, 2), strides=(2, 2)))

# third
add_conv2d(model, third)
add_conv2d(model, third)
add_conv2d(model, third)
add_conv2d(model, third)
model.add(MaxPooling2D(pool_size = (2, 2), strides=(2, 2)))

# fourth
add_conv2d(model, fourth)
add_conv2d(model, fourth)
add_conv2d(model, fourth)
add_conv2d(model, fourth)
model.add(MaxPooling2D(pool_size = (2, 2), strides=(2, 2)))

# fifth
add_conv2d(model, fifth)
add_conv2d(model, fifth)
add_conv2d(model, fifth)
add_conv2d(model, fifth)
model.add(MaxPooling2D(pool_size = (2, 2), strides=(2, 2)))

model.add(GlobalAveragePooling2D())
add_dens(model, 2048)
add_dens(model, 2048)
add_dens(model, 256, activation='softmax')

# InceptionV3
https://arxiv.org/pdf/1512.00567.pdf

In [66]:
model = InceptionV3(include_top=False, weights='imagenet')
image = Input(shape=(150, 150, 3))

x = model(image)
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation="relu")(x)
x = Dense(512, activation="relu")(x)
x = Dense(256)(x)
classes = Activation('softmax')(x)
model = Model(inputs=image, outputs=classes)

for layer in model.layers[:48]:
    layer.trainable = False

In [48]:
model = InceptionV3(include_top=False, weights='imagenet', input_shape=(150, 150, img_channels))
    
x = model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(512, activation="relu")(x)
predictions = Dense(256, activation="softmax")(x)

# creating the final model 
model_final = Model(inputs = model.input, outputs = predictions)

In [67]:
model.count_params()

22327328

In [68]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_24 (InputLayer)        (None, 150, 150, 3)       0         
_________________________________________________________________
inception_v3 (Model)         multiple                  21802784  
_________________________________________________________________
global_average_pooling2d_9 ( (None, 2048)              0         
_________________________________________________________________
dense_22 (Dense)             (None, 256)               524544    
_________________________________________________________________
activation_1417 (Activation) (None, 256)               0         
Total params: 22,327,328
Trainable params: 0
Non-trainable params: 22,327,328
_________________________________________________________________


In [69]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.0004),
              metrics=['accuracy'])

In [70]:
batch_size = 128

dir_transformed = '../output'+str(img_width)
dir_train = os.path.join(dir_transformed, 'train')
dir_val = os.path.join(dir_transformed, 'validation')
dir_test = os.path.join(dir_transformed, 'test')

train_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
    dir_train,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

val_datagen = ImageDataGenerator(rescale=1. / 255)

val_generator = val_datagen.flow_from_directory(
    dir_val,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

test_generator = val_datagen.flow_from_directory(
    dir_test,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    shuffle=False)

Found 19540 images belonging to 256 classes.
Found 2560 images belonging to 256 classes.
Found 7680 images belonging to 1 classes.


In [71]:
callback_1 = TensorBoard(log_dir='./logs', histogram_freq=0, write_graph=False, write_images=False)
callback_2 = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=0, mode='auto')
callback_3 = ModelCheckpoint('model-{epoch:02d}-{val_acc:.3f}-{val_loss:.3f}.hdf5', monitor='val_acc', save_best_only=True, verbose=1)
callback_4 = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.1, patience = 3, min_lr = 0.0001)

In [72]:
%%time
model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=20,
    validation_data=val_generator,
    validation_steps=20,
    callbacks=[callback_1, callback_2, callback_3, callback_4]
)

Epoch 1/20
  1/100 [..............................] - ETA: 1866s - loss: 5.8723 - acc: 0.0000e+00

KeyboardInterrupt: 

In [ ]:
model.summary()

In [4]:
model = load_model('../results/model.hdf5', compile=False)
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.004),
              metrics=['accuracy'])

In [7]:
model.evaluate_generator(val_generator, 
                         steps=len(test_generator.filenames) // batch_size + 1)

[3.6606045042882198, 0.32902151639344263]

In [ ]:
results = model.predict_generator(test_generator, 
                                  steps=len(test_generator.filenames) // batch_size + 1)
results = np.argmax(results, axis=1)+1

# create submission dataframe (in specified kaggle format)
sub = pd.DataFrame(
    data=[row for row in zip([i.split("/")[1] for i in test_generator.filenames], results.astype(int).tolist())],
    columns=['image', 'class']
)

In [ ]:
sub_name = '../results/sub-0.32-3.55.csv'
sub.to_csv(sub_name, encoding='utf-8', index=False)